In [44]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.functional as F
import random
import tqdm
from torch import nn
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
np.set_printoptions(threshold=200)
torch.set_printoptions(edgeitems=200)

In [45]:
import pickle
with open("pickle/pickle_r_all", "rb") as f:
    r = pickle.load(f)
with open("pickle/pickle_p_all", "rb") as f:
    p = pickle.load(f)

In [46]:
# data = r.data_c[r.data_c['date']>'2024-07-01']
data = r.data_c

In [47]:
from Class.LearningDataProcessor import RacingDataProcessor
from Class.Model import HorseRaceModel
from Class.LearningModule import LearningModule
from Func.Deeplearning_func import set_criterion
from Func.Deeplearning_func import log_func
from Func.Deeplearning_func import create_predictionTable
import os

In [48]:
base_path = "unsamble_pickle"
folder_list = os.listdir(base_path)
folder_list

dfs = [pd.read_pickle(base_path+'/'+folder_name) for folder_name in folder_list]
dfs = [df.rename(columns={'pred_0':f'pred_{5*i}',
                  'pred_1':f'pred_{5*i+1}',
                  'pred_2':f'pred_{5*i+2}',
                  'pred_3':f'pred_{5*i+3}',
                  'pred_4':f'pred_{5*i+4}',
                  'rank_0':f'ranking_{5*i}',
                  'rank_1':f'ranking_{5*i+1}',
                  'rank_2':f'ranking_{5*i+2}',
                  'rank_3':f'ranking_{5*i+3}',
                  'rank_4':f'ranking_{5*i+4}',
                  }) for i ,df in enumerate(dfs)]
df_list = data.reset_index()
for i, df in enumerate(dfs):
    pred = [column for column in df.columns if 'pred' in column]
    ranking = [column for column in df.columns if 'ranking' in column]
    df = df[pred+ranking+['date', 'horse_id']]
    df_list = df_list.merge(df, on=['date', 'horse_id'], how='left')

In [49]:
ranking = [column for column in df_list.columns if 'ranking' in column]
df_list['ranking_mean'] =  df_list[ranking].mean(axis = 1)

In [50]:
test =  df_list[['index','着順','人気','単勝','ranking_mean','horse_id','date','ranking_0']]

In [51]:
test2 = test[test['ranking_mean']<1.5]
# test2 = test2[test2['単勝']>8]

In [52]:
len(test2[test2['着順']==1])/len(test2)

0.26501296335225283

In [53]:
len(test['index'].unique())/len(test2)

1.6887394015836312